In [1]:
#import needed packages
import numpy as np
import os 
import matplotlib.pyplot as plt
import cv2
import keras
#import keras
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, Reshape, Flatten, Input, Conv2D, MaxPooling2D, Activation, Dropout, Conv2DTranspose, GlobalAveragePooling2D, BatchNormalization
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Based on architecture from AnimeGAN and Ganyu
# Function to list all images in a given directory
def listImages(basePath, contains=None):
   # Call listFiles function to list all files in the directory
   return listFiles(basePath, contains=contains)

              

In [ ]:

# Function to list all files in a given directory
def listFiles(basePath, contains=None):
    # Iterate through the directory
    for (rootDirectory, directoryNames, filenames) in os.walk(basePath):
        # Iterate through the filenames
        for f in filenames:
            # Check if the filename contains the given string
            if f.find(contains) == -1 and contains is not None:
                # Skip the file if it does not contain the given string
                continue
          
            yield os.path.join(rootDirectory, f).replace(" ", "\\ ")

In [ ]:
# Function to load images from a given directory
def loadImages(directory='', size=(512,512)):
    # Initialize an empty list for labels
    labels, label, numToUse = [], 0, 10000
    # Get a list of image paths
    imagePaths = list(listImages(directory))[:numToUse]
    # Initialize an empty list for images to be stored in
    images = []
    # Iterate through the image paths
    for i in range(len(imagePaths)):
      # Read and resize image, convert the image to RGB
      images.append(cv2.cvtColor(cv2.resize(cv2.imread(imagePaths[i].replace('\\','/')), size), cv2.COLOR_BGR2RGB))
    # Return the list of images
    return images

In [ ]:
# Attribution: 
class GAN():
    def __init__(self):
        self.img_shape = (512, 512, 3)
        self.noise_size = 200
        optimizer = Adam(0.0002,0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        self.combined = Sequential()
        self.combined.add(self.generator)
        self.combined.add(self.discriminator)
        
        self.discriminator.trainable = False
        
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        self.combined.summary()
        
    def build_generator(self):
        noise_shape = (self.noise_size,)
        
        model = Sequential()
        
        model.add(Dense(4*4*512, activation='linear', input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Reshape((4, 4, 512)))
        
        model.add(Conv2DTranspose(1024, kernel_size=[4,4], strides=[2,2], padding="same",
                                  kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
        model.add(LeakyReLU(alpha=0.2))
         
        model.add(Conv2DTranspose(1024, kernel_size=[4,4], strides=[2,2], padding="same",
                                  kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(512, kernel_size=[4,4], strides=[2,2], padding="same",
                                  kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(128, kernel_size=[4,4], strides=[2,2], padding="same",
                                  kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(64, kernel_size=[4,4], strides=[2,2], padding="same",
                                  kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(32, kernel_size=[4,4], strides=[2,2], padding="same",
                                  kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(16, kernel_size=[4,4], strides=[2,2], padding="same",
                                  kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2DTranspose(3, kernel_size=[4,4], strides=[1,1], padding="same",
                                  kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))

        model.add(Activation("tanh"))
        
        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model = Sequential()

        model.add(Conv2D(128, (3,3), padding='same', input_shape=self.img_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(128, (3,3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(3,3)))
        model.add(Dropout(0.2))
        model.add(Conv2D(64, (3,3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(64, (3,3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(3,3)))
        model.add(Dropout(0.3))
        model.add(Conv2D(32, (3,3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(32, (3,3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(3,3)))
        model.add(Dropout(0.3))
        model.add(Flatten())
        model.add(Dense(128))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(64))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity = model(img)
        

        return Model(img, validity)

    def train(self, epochs, batch_size=128, metrics_update=50, save_images=100, save_model=2000):

        X_train = np.array(images)
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        half_batch = int(batch_size / 2)
        
        mean_d_loss=[0,0]
        mean_g_loss=0

        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, self.noise_size))
            gen_imgs = self.generator.predict(noise)
            d_loss = 0.5 * np.add(self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1))),
                                  self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1))))

            for _ in range(2):
                noise = np.random.normal(0, 1, (batch_size, self.noise_size))

                valid_y = np.array([1] * batch_size)
                g_loss = self.combined.train_on_batch(noise, valid_y)
            
            mean_d_loss[0] += d_loss[0]
            mean_d_loss[1] += d_loss[1]
            mean_g_loss += g_loss
            
            if epoch % metrics_update == 0:
                print ("%d [Discriminator loss: %f, acc.: %.2f%%] [Generator loss: %f]" % (epoch, mean_d_loss[0]/metrics_update, 100*mean_d_loss[1]/metrics_update, mean_g_loss/metrics_update))
                mean_d_loss=[0,0]
                mean_g_loss=0
            
            if epoch % save_images == 0:
                self.save_images(epoch)
            
            if epoch % save_model == 0:
                self.generator.save("generator_%d" % epoch)
                self.discriminator.save("discriminator_%d" % epoch)

    def save_images(self, epoch):
        noise = np.random.normal(0, 1, (25, self.noise_size))
        gen_imgs = self.generator.predict(noise)
        
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(5,5, figsize = (30,30))

        for i in range(5):
            for j in range(5):
                axs[i,j].imshow(gen_imgs[5*i+j])
                axs[i,j].axis('off')

        plt.show()
        
        fig.savefig("Boats/Boat_%d.png" % epoch, dpi=200)
        plt.close()

In [ ]:
images=loadImages('/input/boatpics/boats512')
gan=GAN()
gan.train(epochs=15001, batch_size=4, metrics_update=200, save_images=500, save_model=1000)